In [1]:
from scipy.sparse import csr_array

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
ACTIVITY_TRAIN = pd.read_csv('../activity_train.csv', names=['uniprot_id', 'mol_id', 'activity'])
ACTIVITY_TRAIN

,uniprot_id,mol_id,activity
0,O14842,CHEMBL2022243,4
1,O14842,CHEMBL2022244,6
2,O14842,CHEMBL2022245,2
3,O14842,CHEMBL2022246,1
4,O14842,CHEMBL2022247,4
...,...,...,...
135706,Q9Y5Y4,CHEMBL4214909,6
135707,Q9Y5Y4,CHEMBL4218012,2
135708,Q9Y5Y4,CHEMBL4217503,7
135709,Q9Y5Y4,CHEMBL4204359,8


Maybe we should have a matrix like...

```
    , unitprot1 , unitprot2, unitprot3, unitprot4, ...
mol1,     1     ,     5    ,          ,     10    , ...
mol2,           ,     1    ,     6     ,     1    , ...
...

```

In [3]:
def CosSim_Matrix(M):
    norms=np.sqrt(np.sum(M*M, axis=1))
    norms[norms<0.001]=0.001  #this will solve rows or cols without variance
    norms_M = np.outer(norms, norms) # ESTA LINHA FAZ CRASHAR O PC
    VC=M.copy()
    return np.dot(VC, VC.T)/norms_M

def RowCenterMatrix(M):
    V  = M.T
    mat= np.nanmean(V, axis=0)
    VC = V - mat
    VC[np.isnan(VC)]=0
    return VC.T

In [35]:
def create_rankingMatrix(rowLabel, colLabel, df):
    rows = list(set(ACTIVITY_TRAIN[rowLabel])) 
    cols = list(map(lambda v: v.strip(), set(ACTIVITY_TRAIN[colLabel])))
    n_rows = len(rows)
    n_cols = len(cols)

    rows = dict(zip(rows, np.arange(n_rows)))
    cols = dict(zip(cols, np.arange(n_cols)))
    mat = np.zeros((n_rows, n_cols))
    # mat[mat==0]=np.nan
    for rw in df.values:
        mat[rows[rw[0]], cols[rw[1].strip()]]=rw[2]
    return mat, rows, cols

mat, prots, mols=create_rankingMatrix("uniprot_id", "mol_id", ACTIVITY_TRAIN)
mat=mat.T
df_mat = pd.DataFrame(mat)

In [39]:
mols["CHEMBL3593901"]

KeyError: 'CHEMBL3593901'